In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import utils
from torch.utils.data import DataLoader
import tqdm as tqdm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from utils import load_csv, drop_cols, remove_strings, groupedAvg, subsample, normalize
from create_dataset import AnimalDatasetEmbedding, UnpairedEmbeddingsDataset
from generators import  OneHotGenerator, SkipOneHotGenerator, SkipTensorEmbeddingGen, TensorEmbeddingGen, OneHotResNetGenerator
from discriminators import PatchDiscriminator, SampleDiscriminator
import os
import glob
import generators

In [ ]:
PHASES = [1,2,3,4,5]
UNPAIRED = False 
SUPERVISED = True


UNET = True
SKIPCONNECTIONS = False 

RESNET = False

EMBEDDING = True           # Use intervention and phase information during training
if EMBEDDING == True:
    DOWN = False
    BOTTLENECK = True
ONEHOTENCODING = True

PATCH = True                # False = Patch Discriminator, True = Sample Discriminator
SMALLER_TARGET = True       # True = train with target dataset size = PERCENTAGE
PERCENTAGE = 0.5            # percentage of target data


# LR scheduler
MultiStepLR = False
GAMMA = 0.1

ReduceLROnPlateau = False
FACTOR = 0.001
PATIENCE = 2

PolinomialLR = True
power = 1
LR_DECAY_AFTER_EPOCH = 100



DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
LEARNING_RATE = 0.0002 
NUM_WORKERS = 16
NUM_EPOCHS = 200

SIG_A = "AoP"           # Drucksignal Hauptschlagader = Aortendruck
SIG_B = "VADcurrent"    # VAD Strom [A] – Pumpemstrom in Ampere
SIG_C = "VadQ"          # Fluss durch VAD (VAD = Ventrikular assistance device = Pumpe) = Pumpenfluss
SIG_D = "LVP"           # Ventrikeldruck links = Drucksignal der linken Herzkammer
TARGET = "LVtot_kalibriert"  # RVtot_kalibriert existiert auch
source_signals = [SIG_D]
CHANNELS = len(source_signals)
WINDOW = 256

GENERATION_AFTER_EPOCH = NUM_EPOCHS # number of epochs after which the model generates a sample

# Use adversarial loss
LAMBDA_DISC = 0.05
GAN_LOSS = True   # adversarial loss
LAMBDA_GAN = 1.0
# Use cycle consistency loss
CYCLE = False
LAMBDA_CYCLE = 1.0
# Use supervised loss
SUPERVISED = True 
LAMBDA_SUPERVISED = 5.0
# Use Identity loss
IDENTITY = False
LAMBDA_IDENTITY = 10.0

In [ ]:
path = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files" 
csv_files = glob.glob(os.path.join(path, "*.csv"))
  
df = pd.DataFrame()
scaler = StandardScaler() 
# loop over the list of csv files
for f in csv_files:
      
    # read the csv file
    df_temp = pd.read_csv(f, sep=";")
    df_temp = utils.drop_cols(df_temp)
    df_temp = df_temp.dropna()
    df_temp = utils.remove_strings(df_temp)
    df_temp = utils.subsample(df_temp, 10)
    df_temp = utils.normalize(df_temp, scaler, phase1 = True)  
      
    # print the content
    df = pd.concat([df, df_temp], axis=0)

In [ ]:
# select which phases to use
df = df.loc[df['Phasenzuordnung'].isin(PHASES)]
print('Size of the dataset',df.shape)

# print('Size of the dataset with data from phase 1',df.shape)
# print('Size of Phase 1: ', df.loc[df['Phasenzuordnung'] == 1].shape)
# print('Size of Phase 2: ', df.loc[df['Phasenzuordnung'] == 2].shape)
# print('Size of Phase 3: ', df.loc[df['Phasenzuordnung'] == 3].shape)
# print('Size of Phase 4: ', df.loc[df['Phasenzuordnung'] == 4].shape)
# print('SIze of Phase 5: ', df.loc[df['Phasenzuordnung'] == 5].shape)

In [ ]:
df = df.reset_index(drop=True)

for index, row in df.iterrows():
    if row['Phasenzuordnung'] == 1:
        df.at[index, 'intervention'] = 0
    elif row['intervention'] == 10:
        if row['contractility'] == 1.0:
            df.at[index, 'intervention'] = 0      # contractility = 1.0 - could be ignored? - phase 0?
        if row['contractility'] == 3.0:
            df.at[index, 'intervention'] = 5      # contractility = 3.0                                        
        if row['contractility'] == 4.0:
            df.at[index, 'intervention'] = 6      # contractility = 4.0


#remove rows with intervention 2, 4 and 6 -> nearly no data
df = df[df.intervention != 2]
df = df[df.intervention != 4]
df = df[df.intervention != 5]
print(df['intervention'].unique())

In [ ]:
print(len(df['animal'].unique()))
# remove animals with less than 10 data points
df = df.groupby('animal').filter(lambda x: len(x) > 10)
print('Number of animals after removing those with less than 10 data points: ', len(df['animal'].unique()))

## Train and test data

In [ ]:
# select animals 3,4,8,11,17 as test animals
test_animals = [3,4,8,11,17]
print('\nTest animal(s):', test_animals)

all_animals = df['animal'].unique()
# remove test animals from train animals
train_animals =  [x for x in all_animals if x not in test_animals]

# test data
df_test = df[df['animal'].isin(test_animals)]

# change the length of the test data to a multiple of the Window size
df_test = df_test.iloc[:len(df_test) - (len(df_test) % WINDOW)]

# train dataframe with only animals from train_animals
df_train = df[df['animal'].isin(train_animals)]
print('\nDifferent animal IDs after removing those that are in the test dataset: ',len(df_train['animal'].unique()))


print('\nTrain data shape:', df_train.shape)
print('\nTest data shape:', df_test.shape)

In [ ]:
# get random number between 0 and Percentage
import random

random.seed(42)

# get random number between 0 and Percentage (PERCENTAGE refers to the target data set size)
random_number = random.uniform(0, PERCENTAGE)

In [ ]:
if PERCENTAGE == 0.001:
    

    df_target = pd.DataFrame()

    for animal in df_train['animal'].unique():
        for phase in df_train['Phasenzuordnung'].unique():
            df_temp = df_train.loc[(df_train['animal'] == animal) & (df_train['Phasenzuordnung'] == phase)]
            # get random window
            random_number = int(random.uniform(0, len(df_temp)))
            df_temp = df_temp.iloc[random_number:256+random_number]
            df_target = pd.concat([df_target, df_temp], axis=0)
            if df_train.shape[0] >= 100/PERCENTAGE * df_target.shape[0]:
                break

# print(df_train.shape[0] / df_target.shape[0])

In [ ]:
if PERCENTAGE > 0.001:

    df_target = pd.DataFrame()

    for animal in df_train['animal'].unique():
        for phase in df_train['Phasenzuordnung'].unique():
            df_temp = df_train.loc[(df_train['animal'] == animal) & (df_train['Phasenzuordnung'] == phase)]
            df_temp = df_temp.iloc[:int(len(df_temp)*PERCENTAGE)]
            df_target = pd.concat([df_target, df_temp], axis=0)

# print(df_train.shape[0] / df_target.shape[0]) 

In [ ]:
# Generator

if UNET:
    if ONEHOTENCODING and not SKIPCONNECTIONS:
        gen_target = OneHotGenerator(INPUTCHANNELS = CHANNELS, OUTPUTCHANNELS = 1, WINDOWSIZE=WINDOW, Down = DOWN, Bottleneck=BOTTLENECK).to(DEVICE)
        gen_source = OneHotGenerator(INPUTCHANNELS = 1, OUTPUTCHANNELS = CHANNELS, WINDOWSIZE=WINDOW, Down = DOWN, Bottleneck=BOTTLENECK).to(DEVICE)

    if ONEHOTENCODING and SKIPCONNECTIONS:
        gen_target = SkipOneHotGenerator(INPUTCHANNELS = CHANNELS, OUTPUTCHANNELS = 1, WINDOWSIZE=WINDOW, Down = DOWN, Bottleneck=BOTTLENECK).to(DEVICE)
        gen_source = SkipOneHotGenerator(INPUTCHANNELS = 1, OUTPUTCHANNELS = CHANNELS, WINDOWSIZE=WINDOW, Down = DOWN, Bottleneck=BOTTLENECK).to(DEVICE)

    if not ONEHOTENCODING and SKIPCONNECTIONS:
        gen_target = SkipTensorEmbeddingGen(INPUTCHANNELS = CHANNELS, OUTPUTCHANNELS = 1, Down = DOWN, Bottleneck=BOTTLENECK).to(DEVICE)
        gen_source = SkipTensorEmbeddingGen(INPUTCHANNELS = 1, OUTPUTCHANNELS = CHANNELS, Down = DOWN, Bottleneck=BOTTLENECK).to(DEVICE)

    if not ONEHOTENCODING and not SKIPCONNECTIONS:
        gen_target = TensorEmbeddingGen(INPUTCHANNELS = CHANNELS, OUTPUTCHANNELS = 1, Down = DOWN, Bottleneck=BOTTLENECK).to(DEVICE)
        gen_source = TensorEmbeddingGen(INPUTCHANNELS = 1, OUTPUTCHANNELS = CHANNELS, Down = DOWN, Bottleneck=BOTTLENECK).to(DEVICE)

if not UNET:
    if ONEHOTENCODING:
        gen_target = OneHotResNetGenerator(INPUTCHANNELS = CHANNELS, OUTPUTCHANNELS= 1, WINDOWSIZE = WINDOW, blocks=6, Down = False, Bottleneck = True)
        gen_source = OneHotResNetGenerator(INPUTCHANNELS = 1, OUTPUTCHANNELS= CHANNELS, WINDOWSIZE = WINDOW, blocks=6, Down = False, Bottleneck = True)


# Discriminator
if PATCH:
    disc_target = PatchDiscriminator(CHANNELS = 1).to(DEVICE)
    disc_source = PatchDiscriminator(CHANNELS = CHANNELS).to(DEVICE)
if not PATCH:
    disc_target = SampleDiscriminator(CHANNELS = 1).to(DEVICE)
    disc_source = SampleDiscriminator(CHANNELS = CHANNELS).to(DEVICE)

# Optimizers 
opt_disc = torch.optim.AdamW(                                         
    list(disc_source.parameters()) + list(disc_target.parameters()), 
    lr=LEARNING_RATE, 
)
opt_gen = torch.optim.AdamW(
    list(gen_source.parameters()) + list(gen_target.parameters()),
    lr=LEARNING_RATE,
)

# Scheduler
if ReduceLROnPlateau:
    gen_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer = opt_gen,
                                                           factor=FACTOR, patience=PATIENCE, threshold=1e-4,
                                                           min_lr=1e-6,
                                                    )
    disc_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer = opt_disc,
                                                            factor=FACTOR, patience=PATIENCE, threshold=1e-4,
                                                            min_lr=1e-6,
                                                    )
if MultiStepLR:
    gen_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer = opt_gen, milestones=[5,6,7,8], gamma=GAMMA)
                                                        
    disc_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer = opt_disc, milestones=[5,6,7,8], gamma=GAMMA)

if PolinomialLR:
    gen_scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer = opt_gen,
                                                      total_iters = NUM_EPOCHS-LR_DECAY_AFTER_EPOCH, 
                                                      power = power,
                                                    )
    disc_scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer = opt_disc,
                                                       total_iters = NUM_EPOCHS-LR_DECAY_AFTER_EPOCH, 
                                                       power = power,
                                                    )

# losses
l1 = nn.L1Loss() 
mse = nn.MSELoss() 

### Load Checkpoints

In [ ]:
# CHECKPOINT_GEN_target = 'Checkpoints/Supervised/exp_1_all_data_gen_target.pth.tar'
# CHECKPOINT_GEN_source = 'Checkpoints/Supervised/exp_1_all_data_gen_source.pth.tar'

# utils.load_checkpoint(CHECKPOINT_GEN_target, gen_target, opt_gen, LEARNING_RATE)
# utils.load_checkpoint(CHECKPOINT_GEN_source, gen_source, opt_gen, LEARNING_RATE)

### Create the dataset and DataLoader

In [ ]:
if SUPERVISED:
    train_dataset = AnimalDatasetEmbedding(df_target, source_signals, target_name = TARGET,  window_length = WINDOW)
    test_dataset = AnimalDatasetEmbedding(df_test, source_signals, target_name = TARGET, window_length = WINDOW)


# Data loader
loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True,)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True,)

In [ ]:
def discriminator_loss(disc, reals, fakes):
    # calculate how close reals are to being classified as real
    real_loss = mse(disc(reals), torch.ones_like(disc(reals)))
    # calculate how close fakes are to being classified as fake
    fake_loss = mse(disc(fakes), torch.zeros_like(disc(fakes)))
    # return the average of real and fake loss
    return (real_loss + fake_loss) 


# @torch.cuda.amp.autocast()
def get_disc_loss(source, target, disc_source, disc_target, fake_source, fake_target
                    ):
    """
    Return the loss of the discriminator given inputs.
    """
    
    # discriminator loss
    disc_target_loss = discriminator_loss(disc_target, target, fake_target) * LAMBDA_DISC
    disc_source_loss = discriminator_loss(disc_source, source, fake_source) * LAMBDA_DISC
    disc_loss = (disc_source_loss + disc_target_loss) #/ 2

    return disc_loss, disc_source_loss, disc_target_loss


In [ ]:
# training loop
from tqdm.auto import tqdm

for epoch in tqdm(range(NUM_EPOCHS)):

    for source, target, source_phase, source_intervention, target_phase, target_intervention in loader:
        # convert to float16
        source = source.float() # neccessary to prevent error: "Input type (torch.cuda.DoubleTensor) 
        target = target.float() # and weight type (torch.cuda.HalfTensor) should be the same"
    
        # move to GPU
        source = source.to(DEVICE)
        target = target.to(DEVICE)
        source_phase = source_phase.to(DEVICE)
        source_intervention = source_intervention.to(DEVICE)
        target_phase = target_phase.to(DEVICE)
        target_intervention = target_intervention.to(DEVICE)

        #  ------------------------------- #
        #  ----- train discriminators ---- #
        #  ------------------------------- #
        with torch.no_grad():
            fake_target = gen_target(source, source_phase, source_intervention, target_phase, target_intervention).detach()
            fake_source = gen_source(target, source_phase, source_intervention, target_phase, target_intervention).detach()

        d_loss, disc_source_loss, disc_target_loss = get_disc_loss(source, target, disc_source, disc_target, fake_source, fake_target)

        # update gradients of discriminator 
        opt_disc.zero_grad() 
        d_loss.backward()
        opt_disc.step()

        

        # -------------------------------- #
        # ------- train generators ------- #
        # -------------------------------- # 

        g_loss = 0

        g_source_loss = mse(disc_source(fake_source), torch.ones_like(disc_source(fake_source))) 
        g_target_loss = mse(disc_target(fake_target), torch.ones_like(disc_target(fake_target))) 

        g_loss += g_source_loss * LAMBDA_GAN + g_target_loss * LAMBDA_GAN
        
        sup_source_loss = l1(gen_source(target, source_phase, source_intervention, target_phase, target_intervention), source)
        sup_target_loss = l1(gen_target(source, source_phase, source_intervention, target_phase, target_intervention), target)

        g_loss += sup_source_loss * LAMBDA_SUPERVISED + sup_target_loss * LAMBDA_SUPERVISED
    

        opt_gen.zero_grad()
        g_loss.backward()
        opt_gen.step()

    if ReduceLROnPlateau == True:  
        disc_scheduler.step(d_loss)
        gen_scheduler.step(g_loss)

    # scheduler step if epoch > LR_DECAY_AFTER_EPOCH
    if PolinomialLR == True and (epoch+1) >= LR_DECAY_AFTER_EPOCH:
        disc_scheduler.step()
        gen_scheduler.step()

In [ ]:
source_losses = []
target_losses = []

disc_source.eval()  # set discriminator to evaluation mode
disc_target.eval()  # turns off Dropouts Layers, BatchNorm Layers etc
gen_target.eval()
gen_source.eval()

for source, target, source_phase, source_intervention, target_phase, target_intervention in test_loader:                
                    
    source = source.float()
    target = target.float()
    source = source.to(DEVICE)
    target = target.to(DEVICE)
    source_phase = source_phase.to(DEVICE)
    source_intervention = source_intervention.to(DEVICE)
    target_phase = target_phase.to(DEVICE)
    target_intervention = target_intervention.to(DEVICE)

    fake_target = gen_target(source, source_phase, source_intervention, target_phase, target_intervention)
    fake_source = gen_source(target, source_phase, source_intervention, target_phase, target_intervention)

    l1_source = l1(source, fake_source)
    l1_target = l1(target, fake_target)
    source_losses.append(l1_source.item())
    target_losses.append(l1_target.item())

print(f"Average L1 loss of source signals: {np.mean(source_losses)}")
print(f"Average L1 loss of target signals: {np.mean(target_losses)}")

### Generate samples

In [ ]:
#  Generate samples
idx = 0
for source, target, source_phase, source_intervention, target_phase, target_intervention in test_loader:
    if idx == 5:
        break              
                    
    source = source.float()
    target = target.float()
    source = source.to(DEVICE)
    target = target.to(DEVICE)
    source_phase = source_phase.to(DEVICE)
    source_intervention = source_intervention.to(DEVICE)
    target_phase = target_phase.to(DEVICE)
    target_intervention = target_intervention.to(DEVICE)

    fake_target = gen_target(source, source_phase, source_intervention, target_phase, target_intervention)
    fake_source = gen_source(target, source_phase, source_intervention, target_phase, target_intervention)

    l1_source_loss = l1(source, fake_source)
    l1_target_loss = l1(target, fake_target)
                        
    fake_target = fake_target.reshape(-1)
    fake_source = fake_source.reshape(-1)
    source = source.reshape(-1)
    target = target.reshape(-1)

    fig, ax = plt.subplots(2, 1, figsize=(10, 8))
    ax[0].plot(source[:256].cpu().detach().numpy(), label= 'Real source sample')
    ax[0].plot(fake_source[:256].cpu().detach().numpy(), label= 'Generated source sample')
    ax[0].set_xlabel('Signal length')
    ax[0].set_ylabel('Loss')
    ax[0].legend()

    ax[1].plot(target[:256].cpu().detach().numpy(), label= 'Real target sample')
    ax[1].plot(fake_target[:256].cpu().detach().numpy(), label= 'Generated target sample')
    ax[1].set_xlabel('Signal length')
    ax[1].set_ylabel('Loss')
    ax[1].legend()

    print(l1_target_loss, l1_source_loss)

    idx += 1    

### Verify results

In [ ]:
# Verify L1 loss of reconstructed and real signals, per phase and intervention

df_phase_1 = df_test.loc[df_test['Phasenzuordnung'].isin([1])]
df_phase_2 = df_test.loc[df_test['Phasenzuordnung'].isin([2])]
df_phase_3 = df_test.loc[df_test['Phasenzuordnung'].isin([3])]
df_phase_4 = df_test.loc[df_test['Phasenzuordnung'].isin([4])]
df_phase_5 = df_test.loc[df_test['Phasenzuordnung'].isin([5])]

df_int_0 = df_test.loc[df_test['intervention'].isin([0])]
df_int_1 = df_test.loc[df_test['intervention'].isin([1])]
df_int_2 = df_test.loc[df_test['intervention'].isin([2])]
df_int_3 = df_test.loc[df_test['intervention'].isin([3])]
df_int_4 = df_test.loc[df_test['intervention'].isin([4])]
df_int_5 = df_test.loc[df_test['intervention'].isin([5])]
df_int_6 = df_test.loc[df_test['intervention'].isin([6])]

In [ ]:
verify_dataset = AnimalDatasetEmbedding(df_int_6, source_signals, target_name = TARGET, window_length = WINDOW)
verify_loader = DataLoader(verify_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True,)

In [ ]:
source_losses = []
target_losses = []

disc_source.eval()  # set discriminator to evaluation mode
disc_target.eval()  # turns off Dropouts Layers, BatchNorm Layers etc
gen_target.eval()
gen_source.eval()

for source, target, source_phase, source_intervention, target_phase, target_intervention in verify_loader:                
                    
    source = source.float()
    target = target.float()
    source = source.to(DEVICE)
    target = target.to(DEVICE)
    source_phase = source_phase.to(DEVICE)
    source_intervention = source_intervention.to(DEVICE)
    target_phase = target_phase.to(DEVICE)
    target_intervention = target_intervention.to(DEVICE)

    fake_target = gen_target(source, source_phase, source_intervention, target_phase, target_intervention)
    fake_source = gen_source(target, source_phase, source_intervention, target_phase, target_intervention)\
        
    l1_source = l1(source, fake_source)
    l1_target = l1(target, fake_target)
    source_losses.append(l1_source.item())
    target_losses.append(l1_target.item())

print(f"Average L1 loss of source signals: {np.mean(source_losses)}")
print(f"Average L1 loss of target signals: {np.mean(target_losses)}")